# Setting

In [566]:
# Parameter Setting
import pandas as pd

folder_name = 'Research-(D5) Synthesized input model'

time_start = '2016-01-01T00:00:00'
time_end = '2019-12-31T00:00:00'

time_start = pd.to_datetime('2016-01-01T00:00:00')
time_end = pd.to_datetime('2019-12-31T00:00:00')

print(len(feature)-2)

company_list = [
    "Information Technology",
    "Health Care",
    "Financials",
    "Industrials",
    "Consumer Discretionary",
    "Energy",
    "Materials",
    "Communication Services",
    "Utilities",
    "Real Estate",
    "Consumer Staples"
]

process_id = 0  #26

company_name = company_list[process_id]
config_2 = {'input_path': '/content/drive/MyDrive/Colab Notebooks/'+folder_name+'/data/2_'+company_name+'_for_model.csv',
            'save_path': '/content/drive/MyDrive/Colab Notebooks/'+folder_name+'/model_saved/model.ckpt',
            }

7


In [567]:
# Google
from google.colab import drive
drive.mount('/content/drive')

# pip installation
!pip install yfinance

import yfinance as yf

# Basic
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# others
from datetime import datetime, timedelta


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [568]:
def same_seed(seed):
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# Set seed for reproducibility
same_seed(config['seed'])


# Benchmark: Mean of excess return

### (1) Get data

In [569]:
def datetime_func(df):
    # Convert the index to a datetime column
    df['datetime'] = df.index

    # Remove the time zone information
    df['datetime'] = pd.to_datetime(df['datetime']).dt.tz_localize(None).dt.date

    return df

# (1) Today Trend
def day_return(df, n, id):
  # Calculate Numerical
  df['today_return'] = (df['Close'] - df['Open']) / df['Open']

  # Calculate categorical
  df['today_return_cate'] = (df['Close'] - df['Open']).apply(lambda x: 1 if x >= 0 else 0)

  # Convert numerical trend to categorical
  df['today_return_cate'] = df['today_return_cate'].astype('category')

  for days in range(1, n+1):
    df[f'{id}_{days}_day_return'] = (df['Close'].shift(-days) / df['Open'].shift(-1) - 1)
  # for days in range(1, n+1):
  #   df[f'{id}_{days}_day_return_cate'] = df[f'{days}_day_return'].apply(lambda x: 1 if x >= 0 else 0)

  return df


In [570]:
# ids = ["^DJI", "^GSPC", "^NDX", "^IXIC", "^SOX", '^NYA', 'XLK']  # DJSI
ids = ["^GSPC", 'XLK']  # DJSI

df_list = []

for id in ids:
  dji = yf.Ticker(id)
  df = dji.history(period='Max')
  df = datetime_func(df)
  df = day_return(df, n=1, id=id)
  df[(str(id))] = (df['Close']-df['Open']) / df['Open']
  # df = df[[(str(id)), f'{id}_1_day_return', 'datetime']]
  df_list.append(df)

# Merge dataframes
df_index = df_list[0]
for df in df_list[1:]:
  df_index = pd.merge(df_index, df, on='datetime', how='inner')

df_index

,Open_x,High_x,Low_x,Close_x,Volume_x,Dividends_x,Stock Splits_x,datetime,today_return_x,today_return_cate_x,...,Low_y,Close_y,Volume_y,Dividends_y,Stock Splits_y,Capital Gains,today_return_y,today_return_cate_y,XLK_1_day_return,XLK
0,1202.839966,1209.219971,1192.719971,1203.569946,680500000,0.0,0.0,1998-12-22,0.000607,1,...,24.042181,24.243124,300500,0.0,0.0,0.0,-0.011090,0,0.001908,-0.011090
1,1203.569946,1229.890015,1203.569946,1228.540039,697500000,0.0,0.0,1998-12-23,0.020747,1,...,24.325876,24.822323,338300,0.0,0.0,0.0,0.001908,1,-0.003810,0.001908
2,1228.540039,1229.719971,1224.849976,1226.270020,246980000,0.0,0.0,1998-12-24,-0.001848,0,...,24.633191,24.727753,243700,0.0,0.0,0.0,-0.003810,0,-0.012241,-0.003810
3,1226.270020,1231.520020,1221.170044,1225.489990,531560000,0.0,0.0,1998-12-28,-0.000636,0,...,24.656836,24.798677,309300,0.0,0.0,0.0,-0.012241,0,-0.001898,-0.012241
4,1225.489990,1241.859985,1220.780029,1241.810059,586490000,0.0,0.0,1998-12-29,0.013317,1,...,24.585911,24.869595,123800,0.0,0.0,0.0,-0.001898,0,0.001912,-0.001898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6244,4345.229980,4393.569824,4337.540039,4373.200195,3794850000,0.0,0.0,2023-10-17,0.006437,1,...,166.899994,169.460007,7658900,0.0,0.0,0.0,0.008030,1,-0.003269,0.008030
6245,4357.350098,4364.200195,4303.839844,4314.600098,3686030000,0.0,0.0,2023-10-18,-0.009811,0,...,167.029999,167.720001,6431400,0.0,0.0,0.0,-0.003269,0,-0.009680,-0.003269
6246,4321.359863,4339.540039,4269.689941,4278.000000,3969730000,0.0,0.0,2023-10-19,-0.010034,0,...,166.470001,166.759995,9995900,0.0,0.0,0.0,-0.009680,0,-0.016439,-0.009680
6247,4273.850098,4276.560059,4223.029785,4224.160156,4004030000,0.0,0.0,2023-10-20,-0.011627,0,...,163.770004,163.940002,7667700,0.0,0.0,0.0,-0.016439,0,0.007427,-0.016439


### (2) excess return

In [571]:
# n_da_return, excess_return are "targets"

for id in ids:
  df_index[f'excess_return_{id}'] = df_index['XLK_1_day_return'] - df_index[f'{id}_1_day_return']
  df_index[f'excess_return_{id}_cate'] = df_index[f'excess_return_{id}'].apply(lambda x: 1 if x >= 0 else 0)

df = df_index
del df_index

In [572]:
df

,Open_x,High_x,Low_x,Close_x,Volume_x,Dividends_x,Stock Splits_x,datetime,today_return_x,today_return_cate_x,...,Stock Splits_y,Capital Gains,today_return_y,today_return_cate_y,XLK_1_day_return,XLK,excess_return_^GSPC,excess_return_^GSPC_cate,excess_return_XLK,excess_return_XLK_cate
0,1202.839966,1209.219971,1192.719971,1203.569946,680500000,0.0,0.0,1998-12-22,0.000607,1,...,0.0,0.0,-0.011090,0,0.001908,-0.011090,-0.018838,0,0.0,1
1,1203.569946,1229.890015,1203.569946,1228.540039,697500000,0.0,0.0,1998-12-23,0.020747,1,...,0.0,0.0,0.001908,1,-0.003810,0.001908,-0.001962,0,0.0,1
2,1228.540039,1229.719971,1224.849976,1226.270020,246980000,0.0,0.0,1998-12-24,-0.001848,0,...,0.0,0.0,-0.003810,0,-0.012241,-0.003810,-0.011605,0,0.0,1
3,1226.270020,1231.520020,1221.170044,1225.489990,531560000,0.0,0.0,1998-12-28,-0.000636,0,...,0.0,0.0,-0.012241,0,-0.001898,-0.012241,-0.015215,0,0.0,1
4,1225.489990,1241.859985,1220.780029,1241.810059,586490000,0.0,0.0,1998-12-29,0.013317,1,...,0.0,0.0,-0.001898,0,0.001912,-0.001898,0.009868,1,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6244,4345.229980,4393.569824,4337.540039,4373.200195,3794850000,0.0,0.0,2023-10-17,0.006437,1,...,0.0,0.0,0.008030,1,-0.003269,0.008030,0.006542,1,0.0,1
6245,4357.350098,4364.200195,4303.839844,4314.600098,3686030000,0.0,0.0,2023-10-18,-0.009811,0,...,0.0,0.0,-0.003269,0,-0.009680,-0.003269,0.000354,1,0.0,1
6246,4321.359863,4339.540039,4269.689941,4278.000000,3969730000,0.0,0.0,2023-10-19,-0.010034,0,...,0.0,0.0,-0.009680,0,-0.016439,-0.009680,-0.004812,0,0.0,1
6247,4273.850098,4276.560059,4223.029785,4224.160156,4004030000,0.0,0.0,2023-10-20,-0.011627,0,...,0.0,0.0,-0.016439,0,0.007427,-0.016439,0.005850,1,0.0,1


### (3) SMA

In [573]:
# 计算5日SMA
period_list = [
    1,
    2,
    3,
    4,
    5,
    10,
    20,
    120,
    240,
    720,
    1200,
    2400,
    4800
]

sma_period = 5
df['SMA'] = df['excess_return_^GSPC'].rolling(window=sma_period).mean()
# df['SMA'] = 1
df['SMA']

0            NaN
1            NaN
2            NaN
3            NaN
4      -0.007550
          ...   
6244    0.001461
6245    0.000090
6246    0.000882
6247    0.001906
6248         NaN
Name: SMA, Length: 6249, dtype: float64

In [574]:
df['SMA'] = df['SMA'].shift(1)
df['SMA']

0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
          ...   
6244    0.000914
6245    0.001461
6246    0.000090
6247    0.000882
6248    0.001906
Name: SMA, Length: 6249, dtype: float64

In [575]:
df['pred'] = df['SMA'] - df['^GSPC_1_day_return']
df

,Open_x,High_x,Low_x,Close_x,Volume_x,Dividends_x,Stock Splits_x,datetime,today_return_x,today_return_cate_x,...,today_return_y,today_return_cate_y,XLK_1_day_return,XLK,excess_return_^GSPC,excess_return_^GSPC_cate,excess_return_XLK,excess_return_XLK_cate,SMA,pred
0,1202.839966,1209.219971,1192.719971,1203.569946,680500000,0.0,0.0,1998-12-22,0.000607,1,...,-0.011090,0,0.001908,-0.011090,-0.018838,0,0.0,1,NaN,NaN
1,1203.569946,1229.890015,1203.569946,1228.540039,697500000,0.0,0.0,1998-12-23,0.020747,1,...,0.001908,1,-0.003810,0.001908,-0.001962,0,0.0,1,NaN,NaN
2,1228.540039,1229.719971,1224.849976,1226.270020,246980000,0.0,0.0,1998-12-24,-0.001848,0,...,-0.003810,0,-0.012241,-0.003810,-0.011605,0,0.0,1,NaN,NaN
3,1226.270020,1231.520020,1221.170044,1225.489990,531560000,0.0,0.0,1998-12-28,-0.000636,0,...,-0.012241,0,-0.001898,-0.012241,-0.015215,0,0.0,1,NaN,NaN
4,1225.489990,1241.859985,1220.780029,1241.810059,586490000,0.0,0.0,1998-12-29,0.013317,1,...,-0.001898,0,0.001912,-0.001898,0.009868,1,0.0,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6244,4345.229980,4393.569824,4337.540039,4373.200195,3794850000,0.0,0.0,2023-10-17,0.006437,1,...,0.008030,1,-0.003269,0.008030,0.006542,1,0.0,1,0.000914,0.010725
6245,4357.350098,4364.200195,4303.839844,4314.600098,3686030000,0.0,0.0,2023-10-18,-0.009811,0,...,-0.003269,0,-0.009680,-0.003269,0.000354,1,0.0,1,0.001461,0.011495
6246,4321.359863,4339.540039,4269.689941,4278.000000,3969730000,0.0,0.0,2023-10-19,-0.010034,0,...,-0.009680,0,-0.016439,-0.009680,-0.004812,0,0.0,1,0.000090,0.011717
6247,4273.850098,4276.560059,4223.029785,4224.160156,4004030000,0.0,0.0,2023-10-20,-0.011627,0,...,-0.016439,0,0.007427,-0.016439,0.005850,1,0.0,1,0.000882,-0.000695


### (4) Categorize

In [576]:
  df['pred_cate'] = df['pred'].apply(lambda x: 1 if x >= 0 else 0)
  df['pred_cate'] = df['pred_cate'].astype('category')

# train_test_split

## (1) Time Period

In [577]:
# We use index to filter for time periods
df = df[(df['datetime']> time_start) & (df['datetime'] < time_end)]

# Drop datetime after using it
df.drop(columns=['datetime'], inplace=True)
df.shape

<ipython-input-577-c71406b1a572>:2: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  df = df[(df['datetime']> time_start) & (df['datetime'] < time_end)]
<ipython-input-577-c71406b1a572>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['datetime'], inplace=True)


(1005, 30)

## (2) Split

In [578]:
# 1. Set up X, y
X = df['pred_cate']
y = df['excess_return_^GSPC_cate']

In [579]:
# 2. train_test_split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=config['seed'], shuffle=config['shuffle'])
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=config['seed'], shuffle=config['shuffle'])

## (3) Check data

In [580]:
def calculate_class_stats(y):
    class_counts = y.value_counts()
    total_samples = len(y)
    class_ratios = class_counts / total_samples
    return class_counts, class_ratios

# 計算類別數量和比例
train_class_counts, train_class_ratios = calculate_class_stats(y_train)
val_class_counts, val_class_ratios = calculate_class_stats(y_val)
test_class_counts, test_class_ratios = calculate_class_stats(y_test)

# 創建包含數量和比例的 DataFrame
class_stats = pd.DataFrame({
    'Train Count': train_class_counts,
    'Train Ratio': train_class_ratios,
    'Validation Count': val_class_counts,
    'Validation Ratio': val_class_ratios,
    'Test Count': test_class_counts,
    'Test Ratio': test_class_ratios
})

# 打印 DataFrame
print(class_stats)


   Train Count  Train Ratio  Validation Count  Validation Ratio  Test Count  \
1          322     0.533997               113          0.562189         113   
0          281     0.466003                88          0.437811          88   

   Test Ratio  
1    0.562189  
0    0.437811  


In [581]:
# Time period
print('Time Period')
print('From:', time_start)
print('To:', time_end, '\n')

# Sample size
print('Sample size:', X.shape[0])
# print('Feature:', X.columns, '\n')
print('Target:', y.name, '\n')
print('Train: Val: Test =', X_train.shape[0]/X.shape[0], X_test.shape[0]/X.shape[0], X_val.shape[0]/X.shape[0],
      X_train.shape[0], X_test.shape[0], X_val.shape[0])

Time Period
From: 2016-01-01 00:00:00
To: 2019-12-31 00:00:00 

Sample size: 1005
Target: excess_return_^GSPC_cate 

Train: Val: Test = 0.6 0.2 0.2 603 201 201


## Prediction

In [582]:
X_train.value_counts()

0    311
1    292
Name: pred_cate, dtype: int64

In [583]:
print(X_val)
print(y_val)

4887    1
4888    0
4889    1
4890    0
4891    0
       ..
5083    0
5084    1
5085    0
5086    1
5087    0
Name: pred_cate, Length: 201, dtype: category
Categories (2, int64): [0, 1]
4887    1
4888    0
4889    1
4890    1
4891    1
       ..
5083    1
5084    1
5085    0
5086    1
5087    1
Name: excess_return_^GSPC_cate, Length: 201, dtype: int64


In [587]:
def model(text, X, y):
  accuracy = accuracy_score(X, y)
  print(accuracy)
model('Train     ', X_train, y_train)
model('Validation', X_val, y_val)
model('Test      ', X_test, y_test)
model('Overall   ', X, y)


0.4925373134328358
0.3283582089552239
0.48258706467661694
0.4577114427860697


In [589]:
print('=====================================================================================================================')
print('Training Result:')
print(classification_report(X_train, y_train))
print(confusion_matrix(X_train, y_train), '\n')

print('=====================================================')
print('Val Result:')
print(classification_report(X_val, y_val))
print(confusion_matrix(X_val, y_val))

print('=====================================================')
print('Test Result:')
print(classification_report(X_test, y_test))
print(confusion_matrix(X_test, y_test))

print('=====================================================')
print('Overall Result:')
print(classification_report(X, y))
print(confusion_matrix(X, y))

Training Result:
              precision    recall  f1-score   support

           0       0.51      0.46      0.48       311
           1       0.48      0.53      0.50       292

    accuracy                           0.49       603
   macro avg       0.49      0.49      0.49       603
weighted avg       0.49      0.49      0.49       603

[[143 168]
 [138 154]] 

Val Result:
              precision    recall  f1-score   support

           0       0.30      0.26      0.28        99
           1       0.35      0.39      0.37       102

    accuracy                           0.33       201
   macro avg       0.32      0.33      0.33       201
weighted avg       0.33      0.33      0.33       201

[[26 73]
 [62 40]]
Test Result:
              precision    recall  f1-score   support

           0       0.52      0.43      0.47       108
           1       0.45      0.55      0.50        93

    accuracy                           0.48       201
   macro avg       0.49      0.49      0.4